# Pips and Connection 

In [18]:
%pip install mongoengine
%pip install pycoshark
%pip install matplotlib
%pip install fpdf

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

     --------------------------------------- 11.0/11.0 MB 19.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.

  Using cached seaborn-0.12.1-py3-none-any.whl (288 kB)


In [51]:
import re
import matplotlib.pyplot as plt
from mongoengine import connect
from pycoshark.mongomodels import Project, VCSSystem, Commit, FileAction, Hunk, Refactoring, IssueSystem, Issue, IssueComment, MailingList, Message
from pycoshark.utils import create_mongodb_uri_string
from mongoengine.connection import disconnect

# You may have to update this dict to match your DB credentials
credentials = {'db_user': '',
               'db_password': '',
               'db_hostname': 'localhost',
               'db_port': 27017,
               'db_authentication_database': '',
               'db_ssl_enabled': False}

uri = create_mongodb_uri_string(**credentials)

# disconnect(alias='default')
connect('smartshark_2_2', host=uri, alias='default')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

In [52]:
projectsObject = Project.objects().all()

projects =[]

for p in projectsObject:
    if(p.name is not  projects ):
        projects.append(p.name)


In [53]:
issue_type = {}
issue_type_refactoring = {}
for p in projects:
    project = Project.objects(name=p).get()

    if(project is not None):
        

        vcs_system = VCSSystem.objects(project_id=project.id).get()

        num_commits = Commit.objects(vcs_system_id=vcs_system.id).count()

        try: 

            issue_tracker = IssueSystem.objects(project_id=project.id).get()

            for issue in Issue.objects(issue_system_id=issue_tracker.id):
                if issue.issue_type_verified is not None:
                    if(issue.issue_type_verified not in issue_type):
                        issue_type[issue.issue_type_verified] = 1
                        issue_type_refactoring[issue.issue_type_verified] = {}
                    else:
                        issue_type[issue.issue_type_verified] += 1
                    
                    for commit in Commit.objects(linked_issue_ids=issue.id):
                        miner = Refactoring.objects(commit_id=commit.id, detection_tool='rMiner')
                        if miner is not None:
                            for ref in miner:
                                
                                if ref.type not in issue_type_refactoring[issue.issue_type_verified]:
                                    issue_type_refactoring[issue.issue_type_verified][ref.type] = 1
                                else:
                                    issue_type_refactoring[issue.issue_type_verified][ref.type] += 1

        except Exception as e:
            print("No issue tracker for project " + p)
            print(e)
   
print(issue_type_refactoring)

In [ ]:
import pandas as pd

type_refactorings  = ['issue']

for issue in issue_type_refactoring:
    for ref in issue_type_refactoring[issue]:
        if ref not in type_refactorings:
            type_refactorings.append(ref)

df = pd.DataFrame(columns = type_refactorings)

for issue in issue_type_refactoring:
    row = {}
    row['issue']= issue
    for ref in issue_type_refactoring[issue]:
        row[ref] = issue_type_refactoring[issue][ref]
    df = df.append(row, ignore_index=True)
df.set_index('issue', inplace=True)
df.to_csv("issue_type_refactoring.csv")

C:\Users\KleitonEwerton\AppData\Local\Temp\ipykernel_4948\1291054926.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\KleitonEwerton\AppData\Local\Temp\ipykernel_4948\1291054926.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\KleitonEwerton\AppData\Local\Temp\ipykernel_4948\1291054926.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\KleitonEwerton\AppData\Local\Temp\ipykernel_4948\1291054926.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)


,issue,extract_variable,extract_method,parametrize_variable,rename_class,change_variable_type,extract_attribute,rename_method,change_return_type,rename_variable,...,split_variable,move_and_rename_class,merge_variable,split_attribute,push_down_attribute,split_parameter,merge_attribute,extract_subclass,replace_attribute,move_and_rename_attribute
0,documentation,9,15,2,3,161,26,5,8,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,bug,892,1337,97,86,2533,1166,827,1032,1054,...,4,28,10,2,7,5,2,1,2,NaN
2,other,6,14,NaN,3,38,43,5,14,9,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
3,improvement,513,1094,69,148,5257,2290,788,1856,1229,...,2,396,23,8,40,1,6,12,2,2
4,test,42,68,5,60,33,14,68,7,50,...,NaN,19,1,NaN,NaN,NaN,2,NaN,NaN,NaN
5,feature_request,6,72,2,4,38,30,29,22,22,...,NaN,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
6,refactoring,NaN,3,NaN,1,7,2,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Analise